In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string
import math
import heapq


In [2]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []

In [113]:
#######################################importing
def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

#######################################Normalization
def delete_punctuations(doc):
    punctuations = '؟،:؛!»«()[]*,{.}@!?،؛<>#$&!~"\|-_+'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = edited_doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    for i in inverted_indexes:
        if len(i[1])/len(df) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens): 
    for i in range(0, len(tokens)):
        # print(verb_roots)
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        # print("common")
                        break
                if common == False:
                    my_regex = r"\b^"+root[1]+r"\.*"
                    x = re.findall(my_regex, tokens[i])
 
                    if len(x) == 0:  
                        my_regex = r"\b...*"+root[1]+r"\b"
                        x1 = re.findall(my_regex, tokens[i])
                        if len(x1)==0:         
                            # print("root")
                            tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

#######################################tokenization
def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        

        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            if len(token) > 0:
                temp = []
                temp.append(token)
                temp.append(df.id[i])
                tokens.append(temp)
    # tokens.sort()
    return tokens

#######################################create inverted index
def create_inverted_indexes(tokens):
    tokens.sort()
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            if len(token[0])>0:
                temp = []
                temp.append(token_temp)

                (unique, counts) = np.unique(doc_temp, return_counts=True)
                # frequencies = np.array((unique, counts), dtype='i4,f4').T.view(np.recarray)       
                frequencies = []      
                for i in range(len(unique)):
                    temp1 = []
                    temp1.append(unique[i])
                    temp1.append(counts[i])
                    frequencies.append(temp1)
                temp.append(frequencies)
                inverted_indexes.append(temp)
                doc_temp = []
                doc_temp.append(token[1])

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

#######################################calculate weights
def calculate_tfidf(df, inverted_indexes):
    doc_length = np.zeros(len(df)+1)
    len_df = len(df)
    for i in range(len(inverted_indexes)):
        if len(inverted_indexes[i][1]) > 0:
            # print(weighted_inverted_index[i])
            # print(len(inverted_indexes[i][1]))
            idf = math.log(len_df/len(inverted_indexes[i][1]), 10)
            inverted_indexes[i].append(idf)
            # print("idf = ", idf)
            for j in range(len(inverted_indexes[i][1])):
                tf = 1 + math.log(inverted_indexes[i][1][j][1], 10)
                # print("tf = ", tf)
                w = tf*idf
                # print("w : ",w)
                inverted_indexes[i][1][j][1] = w
                # if inverted_indexes[i][1][j][0] == 6545:
                #     print("6545: ", inverted_indexes[i][1][j])
                #     print("6545: ", inverted_indexes[i][0])
                # if inverted_indexes[i][1][j][0] == 6550:
                #     print("6550: ", inverted_indexes[i][1][j])
                #     print("6550: ", inverted_indexes[i][0])


                doc_length[inverted_indexes[i][1][j][0]] = doc_length[inverted_indexes[i][1][j][0]] + np.power(w, 2)
                
    doc_length =  np.sqrt(doc_length)
    return inverted_indexes, doc_length

#######################################create champion list
def create_championList(inverted_indexes, r):
    limit = r
    championList = []
    for i in range(len(inverted_indexes)):
        temp = []
        temp.append(inverted_indexes[i][0])
        temp_postingList = list(inverted_indexes[i][1])
        temp_postingList.sort(key=lambda x: x[1], reverse=True)

        temp1 = []
        r = limit
        if r > len(temp_postingList):
            r = len(temp_postingList)
        for j in range(r):
            # print(temp_postingList[j])
            temp1.append(temp_postingList[j])
        temp.append(temp1)
        if len(inverted_indexes[i]) == 3:
            temp.append(inverted_indexes[i][2])
        championList.append(temp)
    return championList

#######################################Searching
def search_token(inverted_indexes, token_name):
    # for token in inverted_indexes:
    #     if token[0] == token_name:
    #         # print("token*****: ", token[0])
    #         return token[1], token[2]
    if token_name in inverted_indexes:
        return inverted_indexes[token_name][0], inverted_indexes[token_name][1]
    # print("this token not exist in our database")
    return -1, -1

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    if 'topic' in df:
        our_res = p_res[['id', 'rank', 'topic', 'url']]
    else:
        our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    return(our_res)
    

def binary_search(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        docs, idf = search_token(inverted_indexes, tokenized_query[i])
        if docs != -1:
            temp_docs_id = pd.DataFrame(docs)
            docs_id = pd.DataFrame(docs_id.append(temp_docs_id))

    if len(docs_id) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    df_res = print_res(df, docs_id)
    return df_res


def ranked_search(df, inverted_indexes, query, doc_length, k, using_heap):
    res = []
    docs_id = pd.DataFrame()
    try:
        using_class = False
        pattern = "cat\:(.*?)\ "
        topic = re.search(pattern, query).group(1)
        if len(topic) > 0:
            print(topic)
            using_class = True
    except AttributeError:
        topic = re.search(pattern, query)
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)

    doc_score_temp = []
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        # print(docs)


        if docs != -1:
            # print(docs)
            # print(idf)
            w_termInQuery = tf_query*idf
            # print("weight term in q: ", tf_query, idf, w_termInQuery)

            for i in range(len(docs)):
                temp = []
                if using_class == True:
                    if dict_topics[docs[i][0]] == topic:
                        temp.append(docs[i][0])
                        # print("docs[i][1]: ",docs[i][1])
                        temp.append(w_termInQuery*docs[i][1])
                        doc_score_temp.append([docs[i][0], w_termInQuery*docs[i][1]])
                else:
                    temp.append(docs[i][0])
                    # print("docs[i][1]: ",docs[i][1])
                    temp.append(w_termInQuery*docs[i][1])
                    doc_score_temp.append([docs[i][0], w_termInQuery*docs[i][1]])

    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1
    doc_score_temp.sort()
    doc_score = []
    doc_score.append(doc_score_temp[0])
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
        else:
            doc_score.append(doc_score_temp[i])
    
    # for i in range(len(doc_score)):
    #     doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked

    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    res_df = print_res(df, docs_id)
    return res_df

#######################################phase3 Clustering
def k_means(k, iteration):
    #select k centroids randomly
    centroids = []
    randoms = np.random.randint(1, len(inverted_index_perDoc) , size=(k))
    for i in range(len(randoms)):
        centroids.append(inverted_index_perDoc[randoms[i]-1][1])
    # print(centroids)

    max_index_row = 0
    cluster_docs = []       
    for it in range(iteration):
        print(it)
        doc_membership = np.zeros((len(df), k))
        for c in range(len(centroids)):
            print("******c= ",c)
            for term in centroids[c]:
            # for key,value in centroids[c].items():
                # print(dict_inverted_indexes)
                docs, idf = search_token(dict_inverted_indexes, term[0])
                if docs != -1:
                    for doc in docs:
                        doc_membership[doc[0]-1][c] = doc_membership[doc[0]-1][c] + term[1]*doc[1]
        #update centroids

        temp_index_max = max_index_row
        max_index_row = np.argmax(doc_membership, axis=1)
        if np.array_equal(temp_index_max, max_index_row):
            print("centroids not changed")
            cluster_name = np.argmax(doc_membership, axis=1)
            for c in range(len(centroids)):
                cluster_docs.append(np.where(cluster_name == c))

            return centroids, doc_membership, cluster_docs

        centroids = []
        for i in range(k):
            # print("******k= ",i)
            doc_id_perCluster = np.where(max_index_row == i)
            # print(doc_id_perCluster)
            temp = []
            for doc_id in doc_id_perCluster[0]:
                # print(doc_id)
                # print("hey")
                for term in inverted_index_perDoc[doc_id][1]:
                    temp.append(term)
            centroids.append(temp)
        for i in range(k):
            # data_items = centroids[i].items()
            # data_list = list(data_items)
            df_centroid = pd.DataFrame(centroids[i], columns=['term','weight'])
            df_centroid = df_centroid.groupby('term', as_index=False)['weight'].mean()
            centroids[i] = df_centroid.to_numpy()
            # centroids[i] = dict(centroids[i]) 

        print(doc_membership)
        cluster_name = np.argmax(doc_membership, axis=1)
        for c in range(len(centroids)):
            cluster_docs.append(np.where(cluster_name == c))

    return centroids, doc_membership, cluster_docs

def ranked_search_withClustering(df, inverted_indexes, query, doc_length, k, using_heap, b2):
    res = []
    docs_id = pd.DataFrame()
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)
    doc_score_temp = []
    docs_id = np.array
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        similarityToCentroids = np.zeros(len(centroids))
        w_termInQuery = tf_query*idf
        for c in range(len(centroids)):
            if tokenized_query[i][0] in centroids[c]:
                similarityToCentroids[c] = similarityToCentroids[c] + w_termInQuery*centroids[c][tokenized_query[i][0]]
    for b in range(b2):
        # print(similarityToCentroids)
        mostSimilar = np.argmax(similarityToCentroids)
        # print(mostSimilar)
        similarityToCentroids[mostSimilar] = 0

        if b == 0:
            docs_id = cluster_docs[mostSimilar][0]
        else:
            docs_id_temp = cluster_docs[mostSimilar][0]
            docs_id = np.append(docs_id, docs_id_temp)

    # print("*****")
    # print(docs_id)
    # print(len(docs_id))

    # print(similarityToCentroids)
    #search in docs
    doc_score_temp = []
    doc_score = []
    for i in range (0 ,len(tokenized_query)):
        # print(tokenized_query[i][0])
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])

        w_termInQuery = tf_query*idf
        # print(len(docs_id))
        for doc_id in docs_id:
            if tokenized_query[i][0] in inverted_index_perDoc[doc_id][1]:
                # print(doc_id)
                # print(w_termInQuery)
                # print(inverted_index_perDoc[doc_id][1][tokenized_query[i][0]])
                doc_score_temp.append([doc_id + 1, inverted_index_perDoc[doc_id][1][tokenized_query[i][0]] * w_termInQuery])
    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    doc_score_temp.sort()
    # print(doc_score_temp)

    doc_score.append(doc_score_temp[0])
    # print(len(doc_score_temp))
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            # print(doc_score_temp[i][0])
            # print(doc_score[len(doc_score)-1][1])
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
            # print(doc_score[len(doc_score)-1][1])
        else:
            doc_score.append(doc_score_temp[i])


    # for i in range(len(doc_score)):
    #     doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked


    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    df_res = print_res(df, docs_id)
    return df_res


In [4]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [105]:
tokens = tokenize(df)

In [106]:
inverted_indexes = create_inverted_indexes(tokens)
inverted_indexes.pop(0)
inverted_indexes, doc_length = calculate_tfidf(df, inverted_indexes)
#normalize inverted_index
for i in range(len(inverted_indexes)):
    for j in range(len(inverted_indexes[i][1])):
        inverted_indexes[i][1][j][1] = inverted_indexes[i][1][j][1] / doc_length[inverted_indexes[i][1][j][0]]

championList = create_championList(inverted_indexes, 4)
# print(inverted_indexes[1])

##create dictionaries
dict_inverted_indexes = []
for i in range(len(inverted_indexes)):
    temp = []
    temp.append(inverted_indexes[i][0])
    temp.append([inverted_indexes[i][1], inverted_indexes[i][2]])
    dict_inverted_indexes.append(temp)
# inverted_indexes = dict(inverted_indexes)
# print(inverted_indexes['سلام'])
dict_championList = []
for i in range(len(championList)):
    temp = []
    temp.append(championList[i][0])
    temp.append([championList[i][1], championList[i][2]])
    dict_championList.append(temp)

dict_inverted_indexes = dict(dict_inverted_indexes)
dict_championList = dict(dict_championList)

#create inverted_index_perDoc
doc_term = []
for i in range(len(inverted_indexes)):
    for j in range(len(inverted_indexes[i][1])):
        doc_term.append([inverted_indexes[i][1][j][0], inverted_indexes[i][0], inverted_indexes[i][1][j][1]])

doc_term.sort()
# print(doc_term)
inverted_index_perDoc = []
temp = 0
temp = doc_term[0][0]
temp1 = []
for i in range(0, len(doc_term)):
    # print(inverted_index_perDoc[len(inverted_index_perDoc)][0])
    if temp == doc_term[i][0]:
        temp1.append([doc_term[i][1], doc_term[i][2]])
    else:
        inverted_index_perDoc.append([temp, temp1])
        temp = []
        temp1 = []

        temp = doc_term[i][0]
        temp1.append([doc_term[i][1], doc_term[i][2]])
inverted_index_perDoc.append([temp, temp1])

In [107]:
centroids, doc_membership, cluster_docs = k_means(3, 12)
for i in range(len(inverted_index_perDoc)):
    inverted_index_perDoc[i][1] = dict(inverted_index_perDoc[i][1])

for i in range(len(centroids)):
    centroids[i] = dict(centroids[i])

0
******c=  0
******c=  1
******c=  2
[[0.04001389 0.02329522 0.02738133]
 [0.02794589 0.05533207 0.03104033]
 [0.02920601 0.02437369 0.03906508]
 ...
 [0.02495452 0.02042171 0.02620888]
 [0.01563316 0.01225014 0.00050611]
 [0.02608012 0.00713049 0.01064599]]
1
******c=  0
******c=  1
******c=  2
[[0.98477332 0.73656944 0.85232959]
 [0.32333577 0.74633045 0.44392708]
 [0.74707482 0.87021454 0.95952203]
 ...
 [0.95587182 0.88792889 1.3453247 ]
 [0.83112024 0.52151624 0.55442936]
 [0.72714284 0.52834399 0.6427835 ]]
2
******c=  0
******c=  1
******c=  2
[[0.97400641 0.75598187 0.86397991]
 [0.31455189 0.75507084 0.44769246]
 [0.72794316 0.88564431 0.98389311]
 ...
 [0.9292604  0.92514989 1.35897526]
 [0.82471641 0.54407439 0.55545747]
 [0.72000213 0.54699161 0.64963583]]
3
******c=  0
******c=  1
******c=  2
[[0.96418401 0.76441528 0.84990146]
 [0.30786664 0.74526147 0.44332636]
 [0.7083426  0.87800147 0.96883992]
 ...
 [0.92173998 0.94451526 1.35670063]
 [0.82007409 0.54027531 0.5599182

In [132]:
query = "حاملگی"
# query_processing(df, inverted_indexes, query)

# print(ranked_search(df, dict_championList, query, doc_length, k = 10, using_heap = True))
# doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = False)
# print(ranked_search_withClustering(df, dict_inverted_indexes, query, doc_length, k = 10, using_heap = True, b2 = 3))

print(ranked_search(df, dict_inverted_indexes, query, doc_length, k = 10, using_heap = True))
query = "cat:health حاملگی"

print(ranked_search(df, dict_inverted_indexes, query, doc_length, k = 10, using_heap = True))


# print(binary_search(df, dict_inverted_indexes, query))

     id      rank                                                url
8  6997  0.393855  https://www.isna.ir/news/98092015327/انواع-دیا...
9  7000  0.393855  https://www.isna.ir/news/98092015327/انواع-دیا...
6  6873  0.334320  https://www.isna.ir/news/98071410719/دلایل-ابت...
7  6879  0.334320  https://www.isna.ir/news/98071410719/دلایل-ابت...
4  6671  0.271924  https://www.isna.ir/news/98032210328/تولید-کرم...
5  6673  0.271924  https://www.isna.ir/news/98032210328/تولید-کرم...
0  5981  0.267747  https://www.isna.ir/news/99061914723/نکاتی-که-...
1  5989  0.267747  https://www.isna.ir/news/99061914723/نکاتی-که-...
2  6555  0.259982  https://www.isna.ir/news/98010300572/زنان-بارد...
3  6559  0.259982  https://www.isna.ir/news/98010300572/زنان-بارد...
health
     id      rank                                                url
8  6997  0.393855  https://www.isna.ir/news/98092015327/انواع-دیا...
9  7000  0.393855  https://www.isna.ir/news/98092015327/انواع-دیا...
6  6873  0.334320  https://

In [ ]:
s = ['میرو']

print(replaceWithRoot(s))

In [ ]:
# high_idf_tokens = []
# for i in range(len(inverted_indexes)):
#     if len(inverted_indexes[i]) == 3:
#         if inverted_indexes[i][2] > 3.84:
#             high_idf_tokens.append(inverted_indexes[i])
#             print(inverted_indexes[i])
# print(len(high_idf_tokens))

In [ ]:
# df_invertedIndex = []
# for i in range(len(inverted_indexes)):
#     df_invertedIndex.append([inverted_indexes[i][0], pd.DataFrame(inverted_indexes[i][1], columns=['term','weight'])])



In [5]:
df_11k = read_dataset("datasets/", "IR00_3_11k News.xlsx")
df_17k = read_dataset("datasets/", "IR00_3_17k News.xlsx")
df_20k = read_dataset("datasets/", "IR00_3_20k News.xlsx")
df_50k = pd.concat([df_11k, df_17k, df_20k])


In [11]:
df_50k['id']  = range(1, 1 + len(df_50k))
# duplicates_loaded = len(df_50k[df_50k.duplicated(subset=['url', 'content', 'topic'])])
# print(duplicates_loaded)

In [13]:
df_50k = df_50k.reset_index(drop=True)

In [16]:
tokens_50k = tokenize(df_50k)

In [13]:
print(len(tokens_11k))
print(len(tokens_17k))
print(len(tokens_20k))

2900745
7631414
6365851


In [17]:
len(tokens_50k)

16898010

In [18]:
inverted_indexes_50k = create_inverted_indexes(tokens_50k)

In [21]:
len(inverted_indexes_50k)

In [19]:
inverted_indexes_50k[0]

243026

In [20]:
inverted_indexes_50k.pop(0)
inverted_indexes_50k, doc_length_50k = calculate_tfidf(df_50k, inverted_indexes_50k)
#normalize inverted_index
for i in range(len(inverted_indexes_50k)):
    for j in range(len(inverted_indexes_50k[i][1])):
        inverted_indexes_50k[i][1][j][1] = inverted_indexes_50k[i][1][j][1] / doc_length_50k[inverted_indexes_50k[i][1][j][0]]


['', []]

In [22]:
dict_inverted_indexes_50k = []
for i in range(len(inverted_indexes_50k)):
    temp = []
    temp.append(inverted_indexes_50k[i][0])
    temp.append([inverted_indexes_50k[i][1], inverted_indexes_50k[i][2]])
    dict_inverted_indexes_50k.append(temp)
# inverted_indexes = dict(inverted_indexes)
# print(inverted_indexes['سلام'])
# dict_championList = []
# for i in range(len(championList)):
#     temp = []
#     temp.append(championList[i][0])
#     temp.append([championList[i][1], championList[i][2]])
#     dict_championList.append(temp)

dict_inverted_indexes_50k = dict(dict_inverted_indexes_50k)
# dict_championList = dict(dict_championList)

In [23]:
df_50k['topic'].unique()

243025

In [57]:
# df_50k.loc[df_50k['topic'] == 'sports'] = df_50k.loc[df_50k['topic'] == 'sports'].topic.replace('sport')
df_50k['topic'] = df_50k['topic'].replace('sports','sport')
df_50k['topic'] = df_50k['topic'].replace('political','politics')

array(['sport', 'politics', 'economy', 'health', 'culture'], dtype=object)

In [55]:
df_50k

In [56]:
len(dict_inverted_indexes_50k)

,id,content,topic,url
0,1,به گزارش ایسنا، پس از استعفا علی نکویی رئیس شا...,sport,https://www.isna.ir/news/99010100077/حواشی-در-...
1,2,به گزارش ایسنا، امیر محسنی با بیان اینکه این ل...,sport,https://www.isna.ir/news/98122922468/ثبت-نام-ب...
2,3,آرش فرهادیان در گفت و گو با ایسنا، درباره آخری...,sport,https://www.isna.ir/news/99010200541/فرهادیان-...
3,4,به گزارش ایسنا، فدراسیون بین المللی شنا قانونی...,sport,https://www.isna.ir/news/99010200528/فناوری-وی...
4,5,به گزارش ایسنا، فدراسیون جهانی صعودهای ورزشی (...,sport,https://www.isna.ir/news/99010200510/تعویق-زما...
...,...,...,...,...
50056,50057,به گزارش حوزه پارلمانی خبرگزاری فارس، [محمدباق...,politics,https://www.farsnews.ir/news/13990612000494/پی...
50057,50058,به گزارش خبرنگار تشکل‌های دانشگاهی خبرگزاری فا...,politics,https://www.farsnews.ir/news/13990612000397/اع...
50058,50059,به گزارش خبرنگار حوزه دولت خبرگزاری فارس، رضا ...,politics,https://www.farsnews.ir/news/13990612000425/اف...
50059,50060,به گزارش خبرنگار دولت خبرگزاری فارس، محسن حاجی...,politics,https://www.farsnews.ir/news/13990612000435/وز...


In [75]:
%%time
dict_topics = {}
for i in range(len(df):
    query = df.content[i]
    # print(query)
    res = ranked_search(df_50k, dict_inverted_indexes_50k, query, doc_length_50k, k = 9, using_heap = True)
    className = res['topic'].value_counts().idxmax()
    dict_topics[i+1] = className
    if i%1000 == 0:
        print(i)


In [88]:
dict_topics[2500]

5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
Wall time: 18min 30s


In [99]:
import pickle
path = 'files/'
a_file = open(path + "IR_Spring2021_ph12_7k_lablesDict.pkl", "wb")
pickle.dump(dict_topics, a_file)
a_file.close()


'economy'

In [126]:
a_file = open(path + "IR_Spring2021_ph12_7k_lablesDict.pkl", "rb")
output = pickle.load(a_file)
print(output)

In [129]:
dict_topics

alth', 5768: 'health', 5769: 'health', 5770: 'health', 5771: 'health', 5772: 'health', 5773: 'politics', 5774: 'politics', 5775: 'health', 5776: 'health', 5777: 'health', 5778: 'health', 5779: 'health', 5780: 'health', 5781: 'health', 5782: 'health', 5783: 'health', 5784: 'health', 5785: 'health', 5786: 'health', 5787: 'health', 5788: 'health', 5789: 'sport', 5790: 'health', 5791: 'health', 5792: 'health', 5793: 'health', 5794: 'health', 5795: 'health', 5796: 'health', 5797: 'health', 5798: 'health', 5799: 'health', 5800: 'politics', 5801: 'health', 5802: 'health', 5803: 'economy', 5804: 'health', 5805: 'politics', 5806: 'health', 5807: 'politics', 5808: 'politics', 5809: 'health', 5810: 'health', 5811: 'health', 5812: 'health', 5813: 'health', 5814: 'health', 5815: 'health', 5816: 'politics', 5817: 'politics', 5818: 'health', 5819: 'health', 5820: 'health', 5821: 'health', 5822: 'health', 5823: 'economy', 5824: 'health', 5825: 'health', 5826: 'health', 5827: 'economy', 5828: 'health',

In [73]:
df_11k

{1: 'sport', 2: 'sport', 3: 'sport'}

In [73]:
dfkk['id']  = range(10, 10 + len(dfkk))

'economy'

In [78]:
# df_11k
duplicates_loaded = len(df_50k[df_50k.duplicated(subset=['url'])])
print(duplicates_loaded)


In [93]:
x['topic'].value_counts().idxmax()


1313


In [72]:
if 'topic' in df_50k:
    print("heyy")


heyy


In [15]:
cc = df_50k.content

xx = []
for i in range(0, 3):
    doc = cc[i]
    print(delete_punctuations(doc))
    # print(doc)

به گزارش ایسنا پس از استعفا علی نکویی رئیس شاخه آکروباتیک ژیمناستیک ایران به
دلیل اختلافات با رئیس فدراسیون ژیمناستیک دبیر این شاخه هم استعفا داد
آکروباتیک ژیمناستیک ایران که به تازگی موفق شده مجوز حضور بانوان را در عرصه های
بین المللی کسب کند بعد از استعفا رئیس این شاخه دچار حواشی زیادی شده است

کریم محمدزاده که از سال ۱۳۹۴ در این فدراسیون در سمت دبیر شاخه آکروباتیک
ژیمناستیک مشغول به کار بوده است بعد از استعفا نکویی از سمت خود کناره‌گیری
کرد در دوره حضور او به عنوان دبیر در این شاخه ملی پوشان آکروباتیک ایران موفق
شده بودند که هشت مدال آسیایی را کسب کنند

در متن استعفای کریم محمد زاده خطاب به رئیس فدراسیون آمده است

باسلام بدینوسیله اینجانب کریم محمدزاده که از سال ۱۳۹۴ در این فدراسیون در سمت
دبیر شاخه آکروباتیک ژیمناستیک مشغول به تلاش شبانه روزی می‌باشم تقاضای استعفای
خود را از تاریخ ۲۶ اسفند ۱۳۹۸ به حضور محترم اعلام می نمایم

خدا را شکر اینجانب به اتفاق مجموعه همکاران توانستیم نتایج خوبی در این مدت از
فعالیت ها که بر همه آشکار است رقم بزنیم و از رهنمودها و آموزشهای اساتید بزرگ
بهره م